In [1]:
ord("h") # ord function gives you unicode code points, don't support strings for any languages

104

In [2]:
list("hello how are you".encode("utf-8")) #sequence will be to long to feed it to llm, will hinder the context of the llm
# so we come to Byte pair encoding 

[104,
 101,
 108,
 108,
 111,
 32,
 104,
 111,
 119,
 32,
 97,
 114,
 101,
 32,
 121,
 111,
 117]

In [3]:
# Byte Pair encoding is just count the pair of bytes with most occourences and assign them to new token
# this is just to shorten the encoding that we got

In [4]:
text = "Hello, 世界! こんにちは世界。你好，世界! नमस्ते, दुनिया! नमस्ते दुनिया! This text includes English, Chinese, Japanese, and Hindi characters."
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))
print(text)
print("-----")
print(tokens)

Hello, 世界! こんにちは世界。你好，世界! नमस्ते, दुनिया! नमस्ते दुनिया! This text includes English, Chinese, Japanese, and Hindi characters.
-----
[72, 101, 108, 108, 111, 44, 32, 228, 184, 150, 231, 149, 140, 33, 32, 227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129, 161, 227, 129, 175, 228, 184, 150, 231, 149, 140, 227, 128, 130, 228, 189, 160, 229, 165, 189, 239, 188, 140, 228, 184, 150, 231, 149, 140, 33, 32, 224, 164, 168, 224, 164, 174, 224, 164, 184, 224, 165, 141, 224, 164, 164, 224, 165, 135, 44, 32, 224, 164, 166, 224, 165, 129, 224, 164, 168, 224, 164, 191, 224, 164, 175, 224, 164, 190, 33, 32, 224, 164, 168, 224, 164, 174, 224, 164, 184, 224, 165, 141, 224, 164, 164, 224, 165, 135, 32, 224, 164, 166, 224, 165, 129, 224, 164, 168, 224, 164, 191, 224, 164, 175, 224, 164, 190, 33, 32, 84, 104, 105, 115, 32, 116, 101, 120, 116, 32, 105, 110, 99, 108, 117, 100, 101, 115, 32, 69, 110, 103, 108, 105, 115, 104, 44, 32, 67, 104, 105, 110, 101, 115, 101, 44, 32, 74, 97, 112, 97, 110, 101, 115, 

In [5]:
def findmostcommonpair(arr):
    print(len(arr))
    freq = dict()
    ans = (arr[0],arr[1])
    cnt = 0
    for i in range(len(arr)-1):
        key = (arr[i],arr[i+1])
        if (key in freq):
            freq[key] += 1
        else:
            freq[key] = 1

    return max(freq, key = freq.get) # gets you the max freq key
    
#mostfreq = findmostcommonpair(tokens)
#mostfreq

In [6]:
def merge(arr, maxkey, replaceby):
    newarr = []
    i = 0
    while(i < len(arr)):
        if(i==len(arr)-1):
            newarr.append(arr[i])
            i+=1
        else:
            if(arr[i]==maxkey[0] and arr[i+1]==maxkey[1]):
                newarr.append(replaceby)
                i+=2
            else:
                newarr.append(arr[i])
                i+=1
    return newarr

#tokens2 = merge(tokens, mostfreq, 256)
#print(len(tokens), len(tokens2))

In [7]:
vocab_size = 276 #homuch big vacab you want
num_merges = vocab_size - 256 #how much you want to merge, for every merge operation we get a new token

i = 1
merges = {}
reversemerges = {}
originalarr = list(tokens)
while(i <= num_merges):
    newtoken = 256+i
    most_freq = findmostcommonpair(tokens)
    merges[most_freq] = newtoken
    reversemerges[newtoken] = most_freq
    print(f"going to merge {most_freq} to {newtoken}")
    tokens = merge(tokens, most_freq, newtoken)
    i+=1

print(len(tokens), len(originalarr))

203
going to merge (224, 164) to 257
185
going to merge (224, 165) to 258
179
going to merge (44, 32) to 259
174
going to merge (33, 32) to 260
170
going to merge (227, 129) to 261
166
going to merge (257, 168) to 262
162
going to merge (262, 257) to 263
158
going to merge (228, 184) to 264
155
going to merge (264, 150) to 265
152
going to merge (265, 231) to 266
149
going to merge (266, 149) to 267
146
going to merge (267, 140) to 268
143
going to merge (105, 110) to 269
140
going to merge (101, 115) to 270
137
going to merge (268, 260) to 271
135
going to merge (263, 174) to 272
133
going to merge (272, 257) to 273
131
going to merge (273, 184) to 274
129
going to merge (274, 258) to 275
127
going to merge (275, 141) to 276
125 203


In [46]:
# given [257, 256] turn into seq []

vocab = {idx: bytes([idx]) for idx in range(256)}
for key in merges:
    vocab[merges[key]] = vocab[key[0]] + vocab[key[1]]


# def decodesingleint(ids):
#     if ids < 256:
#         return list((ids,))

#     tup = reversemerges[ids]
#     l1 = decodesingleint(tup[0])
#     l2 = decodesingleint(tup[1])
    
#     l1+=l2
#     return l1

# def decode2(ids):
#     newarr = []
#     encodings = {}
#     for i in range(256):
#         encodings[i] = chr(i)

#     for i in ids:
#         newarr += decodesingleint(i)
#     st = ""
#     for i in newarr:
#         st+=encodings[i]
        
#     return st

def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors = 'replace')

    return text
    
decode([275])
# decode2([275])

'नमस�'

In [82]:
def encode(text):
    tokens = list(text.encode("utf-8"))
    for key in merges:
        newarr = []
        i = 0
        while i < (len(tokens)):
            if(i+1<len(tokens) and tokens[i]==key[0] and tokens[i+1]==key[1]):
                newarr.append(merges[key])
                i+=2
            else:
                newarr.append(tokens[i])
                i+=1
        tokens = newarr
        
    return tokens
        
print(decode(encode("This text includes English, Chinese, Japanese, and Hindi characters.")))

This text includes English, Chinese, Japanese, and Hindi characters.


In [ ]:
## video link https://www.youtube.com/watch?v=zduSFxRajkE